# Pipeline de Redirecionamentos
Objetivo: Esse pipeline tem como objetivo demonstrar os passos para redirecionamento de links através da API do Switchy de captura ou vendas em etapas de campanhas da Aloud

# 0. Importação de Dependências

In [15]:
# Importações básicas
import time
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
import urllib.parse
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')


# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# utilitários switchy
from switchy_utils import SwitchyAPI

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

# Inicializar API Switchy
switchy = SwitchyAPI()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


# 1. Load Switchy Links


In [11]:
links_df = switchy.get_links()

# Transformando em Dataframe para melhor manipulação
links_df = pd.DataFrame(links_df)

# 2. Redirecionando páginas de Captura - Rodar em 12/01/2025 na abertura da Live

## 2.1. Filtrando Páginas a Redirecionar

In [4]:
# Filter rows based on column: 'url'
links_df = links_df[links_df['url'].str.contains("pc-ofan-v", regex=False, na=False, case=False)]

## 2.2. Definindo nova URL de destino

In [5]:
# Definindo nova URL base com UTMs legado
NEW_BASE_URL='https://mathjerke.com.br/pv-ofan-v9' # Página de Vendas com checkout Live
links_df['new_url'] = links_df['url'].apply(lambda x: f"{NEW_BASE_URL}?{x.split('?')[-1]}")

/var/folders/12/pylfm93n08bfyhjlzz4j80y80000gn/T/ipykernel_17699/4231508499.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  links_df['new_url'] = links_df['url'].apply(lambda x: f"{NEW_BASE_URL}?{x.split('?')[-1]}")


## 2.3. Alterando Links

In [ ]:
total_links = len(links_df)
alterados = 0

for index, row in links_df.iterrows():
    url = row['url']
    new_url = row['new_url']
    link_id = row['id']

    result = switchy.update_link_url(link_id=link_id, new_url=new_url)

    if result.get('affected_rows', 0) > 0:
        alterados += 1
        print(f"  Sucesso! Links alterados até agora: {alterados} de {total_links}")
    else:
        print("  Nenhum link alterado.")

   

    time.sleep(1)

print(f"\nResumo final: {alterados} de {total_links} links alterados.")

Links id: bf25_l_grupo_v_msg72
URL original: https://mathjerke.com.br/pc-ofan-v1?utm_source=whatsapp&utm_medium=grupos&utm_content=msg72&utm_campaign=lcto-bf25-leads&sck=utm_source=whatsapp%26utm_campaign=lcto-bf25-leads%26utm_content=msg72%26utm_medium=grupos%26campaign_id=lcto-bf25-leads
Nova URL: https://mathjerke.com.br/pv-ofan-v9?utm_source=whatsapp&utm_medium=grupos&utm_content=msg72&utm_campaign=lcto-bf25-leads&sck=utm_source=whatsapp%26utm_campaign=lcto-bf25-leads%26utm_content=msg72%26utm_medium=grupos%26campaign_id=lcto-bf25-leads


{'affected_rows': 1,
 'returning': [{'id': 'bf25_l_grupo_v_msg72',
   'url': 'https://mathjerke.com.br/pv-ofan-v9?utm_source=whatsapp&utm_medium=grupos&utm_content=msg72&utm_campaign=lcto-bf25-leads&sck=utm_source=whatsapp%26utm_campaign=lcto-bf25-leads%26utm_content=msg72%26utm_medium=grupos%26campaign_id=lcto-bf25-leads',
   'deepLinkingEnable': False}]}

# 3. Alterando Links de Checkout

## 3.1. Filtrando links de checkout por oferta e produto

### Padrão dos Links Hotmart
Links da Hotmart seguem a seguinte formato: 

#### https://pay.hotmart.com/M86482747L?off=8x9lqzyk&checkoutMode=10&sck=utm_source=sales%26utm_campaign=sales%26utm_content=pvd%26utm_medium=whatsapp%26seller=tifanywerner


No link de Exemplo podemos identificar como elementos

1. Slug da URL: M86482747L -> Código de Identificação do Produto (É diferente do Product ID na Hotmart)
2. Parâmetro Off: 8x9lqzyk -> Indica o código da Oferta associada ao produto em questão definido pelo Slug
3. Parâmetro CheckoutMode: 10 -> Indica o modo de exibição do Checkout com ou sem a configuração do Checkout personalizado
4. Parâmetro sck: Representa a String encodada com o tracking para a oferta em questão.

### Filtrando Links Hotmart pelo Switch

Normalmente começamos filtrando pelo Slug equivalente ao código do produto para garantir que modificaremos apenas os links referentes ao produto principal.
Posteriormente usando o urlparser identificamos e isolamos apenas os códigos de oferta que queremos redirecionar.
Uma vez isolados os links seguimos com o código de modificação dos links normalmente

In [ ]:
OFFERS_TO_FILTER = ['8x9lqzyk', '8x9lqzyk']
OFFER_CODE_TO_REDIRECT = '8x9lqzyk'

#3.1.1 Filtrando apenas links de checkout Hotmart
links_df = links_df[links_df['url'].str.contains("pay.hotmart.com", regex=False, na=False, case=False)]

# 3.1.2. Filtrando apenas links de produto CDF
links_df = links_df[links_df['url'].str.contains("M86482747L", regex=False, na=False, case=False)]

#3.1.3 Isolando o Código da Oferta
links_df['offer_code'] = links_df['url'].str.extract(r'off=(\w+)')

#3.1.4 Filtrando apenas os links de oferta
links_df = links_df[links_df['offer_code'].isin(OFFERS_TO_FILTER)]

#3.1.5. Se o objetivo for redirecionar os links para outra oferta, podemos usar o seguinte código:

def replace_offer_code(url, new_offer_code):
    parsed = urllib.parse.urlparse(url)
    query = urllib.parse.parse_qs(parsed.query)
    if 'off' in query:
        # substitui o valor de 'off' pela nova oferta
        query['off'] = [new_offer_code]
    else:
        # adiciona 'off' com o valor desejado
        query['off'] = [new_offer_code]
    new_query = urllib.parse.urlencode(query, doseq=True)
    new_url = urllib.parse.urlunparse(parsed._replace(query=new_query))
    return new_url

links_df['new_url'] = links_df['url'].apply(lambda x: replace_offer_code(x, OFFER_CODE_TO_REDIRECT))